(source: https://causalinference.gitlab.io/dowhy/do_why_estimation_methods.html)

In [0]:
#!pip install git+https://github.com/microsoft/dowhy.git

# DoWhy: Different estimation methods for causal inference
This is quick introduction to DoWhy causal inference library. We will load in a sample dataset and use different methods for estimating causal effect from a (pre-specified) treatment variable to a (pre-specified) outcome variable.

In [0]:
import numpy as np
import pandas as pd
import logging

import dowhy
from dowhy.do_why import CausalModel
import dowhy.datasets

Let us first load a dataset. For simplicity, we simulate a dataset with linear relationships between common causes and treatment, and common causes and outcome.

Beta is the true causal effect.

In [0]:
data = dowhy.datasets.linear_dataset(beta=10,
        num_common_causes=5,
        num_instruments = 2,
        num_samples=10000,
        treatment_is_binary=True)
df = data["df"]

In [0]:
data

Note that we are using a pandas dataframe to load the data.

## Identifying the causal estimand
We now input a causal graph in the DOT graph format.

In [0]:
# With graph
model=CausalModel(
        data = df,
        treatment=data["treatment_name"],
        outcome=data["outcome_name"],
        graph=data["gml_graph"],
        instruments=data["instrument_names"],
        logging_level = logging.INFO
        )

In [0]:
model.view_model()

We get a causal graph. Now identification and estimation is done.



In [0]:
identified_estimand = model.identify_effect()
print(identified_estimand)

### Method 1: Regression
Use linear regression.

In [0]:
causal_estimate_reg = model.estimate_effect(identified_estimand,
        method_name="backdoor.linear_regression",
        test_significance=True)
print(causal_estimate_reg)
print("Causal Estimate is " + str(causal_estimate_reg.value))

### Method 2: Stratification
We will be using propensity scores to stratify units in the data.

In [0]:
causal_estimate_strat = model.estimate_effect(identified_estimand,
        method_name="backdoor.propensity_score_stratification")
print(causal_estimate_strat)
print("Causal Estimate is " + str(causal_estimate_strat.value))

###Method 3: Matching
We will be using propensity scores to match units in the data.

In [0]:
causal_estimate_match = model.estimate_effect(identified_estimand,
        method_name="backdoor.propensity_score_matching")
print(causal_estimate_match)
print("Causal Estimate is " + str(causal_estimate_match.value))

### Method 4: Weighting
We will be using (inverse) propensity scores to assign weights to units in the data.

In [0]:
causal_estimate_ipw = model.estimate_effect(identified_estimand,
        method_name="backdoor.propensity_score_weighting")
print(causal_estimate_ipw)
print("Causal Estimate is " + str(causal_estimate_ipw.value))